## **OpenMP e GPU no Google Colab**

Para utilizar OpenMP no Google Colab, inicialmente escreva seu código numa célula de código e, na primeira linha, utilize o comando %%writefile com o nome do arquivo a ser gravado no seu diretório de trabalho.

In [1]:
%%writefile main.cpp

#include <iostream>
int main()
{
    #pragma omp parallel
    {
        std::cout << "Hello World\n";
    }
    return 0;
}

Writing main.cpp


Em seguida, você deve compilar o seu código:

In [2]:
!g++ -fopenmp -o main main.cpp

Para executar o seu código, basta utilizar o nome do executável:

In [3]:
!./main

Hello World
Hello World


**EXERCÍCIO.** Considere o código abaixo e sua medida de tempo:

In [4]:
%%writefile main.cpp

#include <iostream>
#define ARRAY_SIZE 100000000
#define ARRAY_VALUE 1231
int main()
{
    int *arr = new int[ARRAY_SIZE];
    std::fill_n(arr, ARRAY_SIZE, ARRAY_VALUE);
    for(int i = 0; i < ARRAY_SIZE; i++)
    {
        // do some relatively long operation
        arr[i] = 3*arr[i]  + arr[i] / 5 - 14;
    }
    return 0;
}

Overwriting main.cpp


In [5]:
!g++ -o main main.cpp

In [6]:
!time ./main


real	0m0.744s
user	0m0.573s
sys	0m0.164s


Utilizando OpenMP, paralelize este código e refaça a medida de tempo:

In [50]:
%%writefile main.cpp

#include <iostream>
#define ARRAY_SIZE 100000000
#define ARRAY_VALUE 1231
int main()
{
    int *arr = new int[ARRAY_SIZE];
    // std::fill_n(arr, ARRAY_SIZE, ARRAY_VALUE);
    #pragma omp parallel for
    for(int i = 0; i < ARRAY_SIZE; i++)
    {
        // do some relatively long operation
        arr[i] = 3*ARRAY_VALUE  + ARRAY_VALUE / 5 - 14;
    }
    return 0;
}

Overwriting main.cpp


In [51]:
!g++ -fopenmp -o main main.cpp

In [52]:
!time ./main


real	0m0.272s
user	0m0.291s
sys	0m0.208s


Para utilizar GPU no Google Colab,  você também deve escrever seu código numa célula de código e, na primeira linha, utilize o comando %%writefile com o nome do arquivo a ser gravado no seu diretório de trabalho.

In [53]:
%%writefile main.cu

#include <thrust/device_vector.h>
#include <iostream>

int main(void)
{
    // inicializa todos os 10 elementos de v na GPU com o valor 1
    thrust::device_vector<int> v(10, 1);
 
    // mostra todos os elementos alocados
    for(int i = 0; i < v.size(); i++)
        std::cout << v[i] << "\n";

    return 0;
}

Writing main.cu


Em seguida, compilamos o arquivo com o nvcc indicando o tipo específico de gpu do Google Colab (sm_37):

In [54]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o main main.cu

Para executar o programa, primeiro você deve ativar o dispositivo GPU no menu Editar--> Configurações de Notebook --> Acelerador de Hardware (GPU). **ATENÇÃO: DEIXE A GPU ATIVADA SOMENTE ENQUANTO FOR EXECUTAR O PROGRAMA. HÁ UM TEMPO LIMITADO DE USO DAS GPUs NO GOOGLE COLAB.**

In [55]:
!./main

1
1
1
1
1
1
1
1
1
1


**EXERCÍCIO.** Considere o código C++ abaixo que calcula a norma de um vetor de 4 componentes e suas medidas de tempo:

In [77]:
%%writefile main.cpp
#include <iostream>
#include <cmath>

int main(void)
{
    // inicializa o vetor 
    float x[4] = {1.0, 2.0, 3.0, 4.0};
    float sum=0.0;
    for (int i=0;i<4;i++)
        sum+=x[i]*x[i];
    std::cout << std::sqrt(sum);
    return 0;
}

Overwriting main.cpp


In [78]:
!g++ -o main main.cpp

In [79]:
!time ./main

5.47723
real	0m0.005s
user	0m0.001s
sys	0m0.002s


Paralelize o código acima em GPU com THRUST e faça uma estimativa de tempo de sua execução:

In [83]:
%%writefile main.cu
#include <iostream>
#include <cmath>
#include <thrust/random.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/iterator/counting_iterator.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/sequence.h>

struct pow_ {

    __host__ __device__
    float operator()(const float& a) {
            return a*a;
    }
};

int main(void)
{
    // inicializa o vetor 
    thrust::device_vector<float> x(5);
    thrust::sequence(x.begin(), x.end());
    thrust::device_vector<float> res(4);
 
    float sum;
    thrust::transform(x.begin()+1, x.end(), res.begin(), pow_());
    sum = thrust::reduce(res.begin(), res.end(), float(0.0));
    std::cout << std::sqrt(sum);
    return 0;
}

Overwriting main.cu
Overwriting main.cu


In [84]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o main main.cu

In [85]:
!time ./main

5.47723
real	0m0.577s
user	0m0.308s
sys	0m0.256s
5.47723
real	0m0.575s
user	0m0.320s
sys	0m0.245s
